# A* Algorithm

## Pramble

**A\*** is a heuristic search algorithm for increasing the efficiency of Djikstra’s algorithm for pathfinding and graph traversal. Here heuristic information means to find a suitable or an intuitive function that helps us to find the shortest path between 2 nodes in a graph, efficiently. It is used in gaming applications especially to design the movements of agents when they are aiming to reach a goal. It is used in network connection systems for sending packets between routers, cost effectively. It also helps robots in navigation$^{[1]}$. It uses the heuristic cost of greedy best first search algorithm and exact cost of Dijkstra's algorithm to find the shortest path. It calculates the heuristic distance from node **n** to the end node and exact cost from start node to **n**.
<p><center>$Distance_{new}(start,v) = Heuristic(v,end) + Distance_{old}(start,v)$</center></p>

$Distance_{new}(start,v)$ is the new updated distance between start node and node **v**, $Distance_{old}(start,v)$ is the exact cost of reaching node **v** from start node and $Heuristic(v,end)$ is the heuristic cost of reaching to end node from **v**. After updating the distance in the graph, we will calculate the shortest path.

<h4>Navigation through Maps</h4>
<p>For finding optimal path between 2 cities or places, we can use A* search algorithm to calculate the shortest path between them. Here the heuristic function could be the air distance(euclidean distance) or great-circle distance between 2 cities.$^{[2]}$</p>

<h4>Video Games</h4>
Pathfinding is a crucial part in Video Games. A* is one of the widely used algorithms in games especially when it involves path-finding problems such as moving characters from one grid point to another grid point. Due to the exponential growth in complexity of the games, researchers focused on improving the A* algorithm to meet the demands of modern games. Many of the optimizations include improving heuristic functions, map representations(search space), introducing new data structures and reducing memory consumptions.$^{[3]}$

![title](fig1.png)
<p>Fig 1: Screenshot from civilization V game, where it uses hexagonal tiles as search space$^{[3]}$</p>

## Short history

[talk about the shakey robot, why this algorithm was needed, and what it is used for today]

## Core
<h3>Other Search Algorithms</h4>

<h5>Dijkstra's Algorithm$^{[4]}$</h5>
<p>An algorithm to find the shortest path in a graph with positive edge weights. It maintains a set of visited and unvisited vertices. It moves to next unvisited node from a node and update the distance if its lower than the preivous. This process continue until the destination vertex is reached.</p>

<h5>Greedy Best First Search$^{[5]}$</h5>
<p>A pathfinding algorithm that works by utilising heuristic function to determine cost effective paths. It evaluates the cost of each path from the current node to the end node and selects the low cost path. This process is repeated until the end node is reached.</p>

<h5>Bellman-Ford Algorithm$^{[6]}$</h5>
<p>It is a shortest path algorithm which works well in negative weighted edges. It updates the shortest distance of node, if a shorter path is found through another node.</p>

<h5>Floyd-Warshall Algorithm$^{[7]}$</h5>
<p>It's the shortest path finding algorithm for all pairs of vertices. For reducing time complexity, it uses Dynamic Programming approach to find the shortest path between vertices, if possible. It doesn’t work for negative cycles, where your sum of edges in a cycle is less than zero.
</p>
<h3>Comparisons</h3>
<p>For understanding how A* algorithm differs from above mentioned path finding algorithms, we are using certain features to differ them, such as:</p>
<p>
    <li><b>Optimization</b>: Discuss on what area algorithm is optimized for.$^{[4]}$
        <ol>
        <li><b>Greedy Best First Search</b>: Finding shortest path between single source node and all other nodes based on heuristic function that depends on current node and end node.</li>
        <li><b>Dijkstra's Algorithm</b>: Finding shortest path between single source node and all other nodes with positive weighted edges.</li>
        <li><b>Bellman-Ford Algorithm</b>: Finding shortest path between single source node and all other nodes including negative weighted edges.</li>
        <li><b>Floyd-Warshall Algorithm</b>: Finding shortest path between all pairs of nodes with only positive weighted edges.</li>
        <li><b>A* Algorithm</b>: Finding shortest path between start and end nodes by reducing search space</li>
        </ol>
    </li>
    <li><b>Relaxation</b>: Discuss on how weights are updated in each iteration.$^{[4]}$
        <ol>
        <li><b>Greedy Best First Search</b>: It finds the node having the smallest distance in a greedy approach and updates the distance of its neighbours.</li>
        <li><b>Dijkstra's Algorithm</b>: It finds the node having the smallest distance based on the actual cost and updates the distance of its neighbours.</li>
        <li><b>Bellman-Ford Algorithm</b>: It considers all possible paths to nodes and updates the distance of each node.</li>
        <li><b>Floyd-Warshall Algorithm</b>: It considers all possible paths to nodes and updates the distance of each node.</li>
        <li><b>A* Algorithm</b>: It chooses the node having minimal sum of distance from start node and heuristic distance from end node. Weights are updated after adding heuristics.</li>
        </ol>
    </li>
    <li><b>Time Complexity</b>: Discuss about the time taken by algorithm to complete. For below, $E$ stands for edges in the graph and $V$ stands for vertices in the graph.$^{[4]}$
        <ol>
        <li><b>Greedy Best First Search</b>: It depends on heuristic function.</li>
        <li><b>Dijkstra's Algorithm</b>: Average is <b>$O(E \times log(V))$</b>, worst is <b>$O(V^2)$</b>. </li>
        <li><b>Bellman-Ford Algorithm</b>: It has a time complexity of <b>$O(VE)$</b>.</li>
        <li><b>Floyd-Warshall Algorithm</b>: It has time complexity of $O(V^3)$.</li>
        <li><b>A* Algorithm</b>: Time complexity depends on heuristic function.</li>
        </ol>
    </li>
    <li><b>Search Techniques</b>: Discuss about the search techniques such as the parameters involved in updating weight functions.$^{[4]}$
        <ol>
        <li><b>Greedy Best First Search</b>: It uses heuristic function to reach to the end node.</li>
        <li><b>Dijkstra's Algorithm, Bellman-Ford Algorithm, Floyd-Warshall Algorithm</b>: It uses the weights initialized in the graph for calculating the shortest path.</li>
            <li><b>A* Algorithm</b>: It uses weights initialised and heuristic function to guide the search to end node</li>
        </ol>
    </li>
</p>

## Demo
[Try to visualize the algorithm]

## Variants
<h4>Heirarchical Pathfinding A* (HPA*) .$^{[3]}$</h4>
<p>Using hierarchical A*, we can reduce the search space hierarchically. For example, if we are going to Blackrock, Dublin from Kinsale, Cork (as shown in Fig 2.), by using A* implementation we will find all the cities in each county as nodes and find its heuristic function value to compute the shortest path between these two. If we use hierarchical A* algorithm, we can reduce the size of search space by considering its counties as its nodes, not the cities. First travel to the neighbourhood of the starting node i.e. boundary of Cork County. Then find the neighbourhood of the end node which is Dublin. After reaching the neighbourhood, travel to the end node i.e. Blackrock. As you can see we have eliminated the cities(nodes) that are not part of Cork and Dublin, and thus reduced search space dramatically resulting in higher efficiency. </p>

![title](fig2.png)
<p>Fig 2: Red lined circles can be denoted as boundaries of start and end nodes</p>


## Conclusion
[Summarize everything here]